In [1]:
import json
import pandas as pd

In [2]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
train_pairs

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289
2,1.0,53602615,587809782
3,1.0,53888651,89598677
4,0.0,56930698,551526166
...,...,...,...
306535,0.0,817327230,822083612
306536,0.0,817560551,818069912
306537,0.0,817854719,817857267
306538,0.0,820036017,820037019


In [3]:
columns = ['variantid', 'categories']
train_data = pd.read_parquet('./datasets/train_data.parquet', columns=columns).set_index('variantid')
train_data['categories'] = train_data['categories'].apply(lambda x: json.loads(x))
train_data['cat2'] = train_data['categories'].apply(lambda x: x['2'])
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=columns).set_index('variantid')
test_data['categories'] = test_data['categories'].apply(lambda x: json.loads(x))
test_data['cat2'] = test_data['categories'].apply(lambda x: x['2'])

In [4]:
train_data

,categories,cat2
variantid,,
51195767,"{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",Электроника
53565809,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",Электроника
56763357,"{'1': 'EPG', '2': 'Электроника', '3': 'Оптичес...",Электроника
56961772,"{'1': 'EPG', '2': 'Электроника', '3': 'Смартфо...",Электроника
61054740,"{'1': 'EPG', '2': 'Электроника', '3': 'Штативы...",Электроника
...,...,...
820128810,"{'1': 'EPG', '2': 'Электроника', '3': 'Расходн...",Электроника
821135769,"{'1': 'EPG', '2': 'Электроника', '3': 'Защитны...",Электроника
822095690,"{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",Электроника


In [5]:
test_data

,categories,cat2
variantid,,
51201254,"{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",Электроника
77151532,"{'1': 'EPG', '2': 'Электроника', '3': 'Запчаст...",Электроника
89664856,"{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",Электроника
90701982,"{'1': 'EPG', '2': 'Электроника', '3': 'Акустик...",Электроника
92484118,"{'1': 'EPG', '2': 'Электроника', '3': 'Батарей...",Электроника
...,...,...
702785891,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",Электроника
704096517,"{'1': 'EPG', '2': 'Электроника', '3': 'Зарядны...",Электроника
705874953,"{'1': 'EPG', '2': 'Электроника', '3': 'Операти...",Электроника


In [6]:
len(test_data) / (len(train_data) + len(test_data))

0.0725050883433815

In [7]:
len(set(test_data.index) & set(train_data.index)), \
len(set(test_data.index) & set(train_data.index)) / len(set(test_data.index) | set(train_data.index))

(3021, 0.006168176212605049)

In [8]:
train_data['cat2'].unique(), test_data['cat2'].unique()

(array(['Электроника', 'Автотовары', 'Хобби и творчество',
        'Строительство и ремонт', 'Спорт и отдых', 'Канцелярские товары',
        'Бытовая химия', 'Дом и сад', 'Детские товары', 'Бытовая техника',
        'Товары для взрослых', 'Галантерея и украшения',
        'Товары для животных'], dtype=object),
 array(['Электроника'], dtype=object))

In [9]:
another_cat_mask = (train_data.loc[train_pairs.variantid1, 'cat2'] != 'Электроника').values | (train_data.loc[train_pairs.variantid2, 'cat2'] != 'Электроника').values 
another_cat_mask.sum()

234

Category 2 is useless

In [10]:
from sklearn.model_selection import train_test_split

# 0.1634 - 70/30%
# 0.1835 - 66/33%
train_variants, val_variants = train_test_split(train_data.index, test_size = 0.1835, random_state=56)
len(train_variants), len(val_variants)

(373191, 83872)

In [11]:
train_idx = train_pairs.variantid1.isin(train_variants) & train_pairs.variantid2.isin(train_variants)
val_idx = ~train_idx
train_idx.sum() / len(train_pairs), val_idx.sum() / len(train_pairs)

(0.6667188621387095, 0.33328113786129054)

In [12]:
variants_in_train = set(train_pairs[train_idx].variantid1) | set(train_pairs[train_idx].variantid2)
variants_in_val = set(train_pairs[val_idx].variantid1) | set(train_pairs[val_idx].variantid2)
len(variants_in_train & variants_in_val), \
len(variants_in_train & variants_in_val) / len(variants_in_train | variants_in_val)

(28328, 0.06202202123304017)

random: 0.108

this: 0.062

In [13]:
train_idx.to_csv('./datasets/train_idx.csv')
val_idx.to_csv('./datasets/val_idx.csv')

how use:
```Py
val_pairs = train_pairs[pd.read_csv('./datasets/val_idx.csv', index_col=0).values].copy()
train_pairs = train_pairs[pd.read_csv('./datasets/train_idx.csv', index_col=0).values].copy()
```